# Walk Forward: A Realistic Approach to Backtesting

In [ ]:
???

![](<src/10_Table_Validation Methods.png>)

## Load the data

In [1]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', parse_dates=['Date'], index_col=0)
df = df.drop(columns='change_tomorrow_direction')
df

,Open,High,Low,Close,Volume,change_tomorrow
Date,,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800,1.549143
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.321692
...,...,...,...,...,...,...
2023-03-14,256.750000,261.070007,255.860001,260.790009,33620300,1.751806
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,3.895731


## Walk Forward Validation

### How `TimeSeriesSplit` works

In [ ]:
list_df_train = []
list_df_test = []

for index_train, index_test in ts.split(df):
    list_df_train.append(df.iloc[index_train])
    list_df_test.append(df.iloc[index_test])

In [ ]:
list_df_train[0]

In [ ]:
list_df_test[0]

In [ ]:
list_df_train[1]

In [ ]:
list_df_test[1]

## Machine Learning Model

### Separate the data

1. Target: which variable do you want to predict?
2. Explanatory: which variables will you use to calculate the prediction?

In [ ]:
y = df.change_tomorrow
X = df[['Open','High','Low','Close','Volume']]

In [ ]:
list_df_train = []
list_df_test = []

for index_train, index_test in ts.split(df):
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]


### Simulate one computation of the ML model

- Compute the model
- Calculate predictions on the test set
- Evaluate how good the model is

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

model_dt = DecisionTreeRegressor(max_depth=15, random_state=42)
model_dt.fit(X_train, y_train)

y_pred = model_dt.predict(X_test)
error_mse = mean_squared_error(y_test, y_pred)
error_mse

### Add the procedure inside the for loop

In [ ]:
model_dt = DecisionTreeRegressor(max_depth=15, random_state=42)

error_mse_list = []

for index_train, index_test in ts.split(df):
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]
    
    model_dt.fit(X_train, y_train)
    
    y_pred = model_dt.predict(X_test)
    error_mse = mean_squared_error(y_test, y_pred)
    
    error_mse_list.append(error_mse)

In [ ]:
error_mse_list

In [ ]:
import  numpy as np

In [ ]:
np.mean(error_mse_list)

## Walk Forward evaluation in backtesting

### Create a new strategy

In [ ]:
from backtesting import Backtest, Strategy

In [ ]:
bt = Backtest(df, ???, cash=10000, commission=.002, exclusive_orders=True)

In [ ]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = DecisionTreeRegressor(max_depth=15, random_state=42)
        self.already_bought = False
        
        X_train = ???
        y_train = ???
        
        self.model.fit(X=X_train, y=y_train)

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create a library of strategies

### Import the strategy and perform the backtest

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import strategies

In [ ]:
from backtesting import Backtest
bt = Backtest(df, ???, cash=10000, commission=.002, exclusive_orders=True)

In [ ]:
import multiprocessing as mp
mp.set_start_method('fork')

In [ ]:
stats_skopt, heatmap, optimize_result = bt.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff

### Interpret the strategy's performance

In [ ]:
bt.plot(filename='reports/walk_forward.html')

## Anchored Walk Forward

## How to improve the strategy?

Adding new conditions

![](<src/10_Table_Backtesting Adjustment Parameters.png>)

![](<src/10_Table_Stop Loss.png>)